In [1]:
import requests
import pandas as pd
import numpy as np

### Gettting the wikipedia page.

In [2]:
Toronto_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
# Now that we have the url where our data from Toronto is, we use BeautifulSoup to extract our data.
from bs4 import BeautifulSoup
soup = BeautifulSoup(Toronto_url,'lxml')



### Finding where our table with Boroughs and Neighborhoods starts

In [3]:
Toronto_BN = soup.find('table',{'class':'wikitable sortable'})


### Creating 3 lists with all Postal Codes, Borough and Neighborhoods

In [4]:
PostalCodes = []
Borough =[]
Neighborhood=[]
for row in Toronto_BN.findAll("tr"):
    cells = row.findAll("td")
    #For each "tr", assign each "td" to a variable.
    if len(cells) == 3:
        PostalCodes.append(cells[0].find(text=True))
        Borough.append(cells[1].find(text=True))
        Neighborhood.append(cells[2].find(text=True))


### Creating my Data Frame

In [5]:
Toronto_df = pd.DataFrame({'PostalCode': PostalCodes,'Neighborhood':Neighborhood,'Borough': Borough})


### Cleaning up table to adjust it as requested (Dropping the "Not assigned" Borough and One Postal Code per Borough and all Neighborhood in it )

In [6]:
# Dropping all rows where Borough is equal to 'Not assigned'
df_Toronto = Toronto_df[Toronto_df.Borough != 'Not assigned']
df_Toronto.reset_index(inplace=True)
df_Toronto = df_Toronto.replace('\n','', regex=True) # This removes all new lines ('\n') that I may have in all columns.
df_Toronto.drop('index',axis=1,inplace=True)


In [7]:
## Assigning the name of the Borough to neigborhoods with value 'Not assigned'
ind=[]
a = df_Toronto['Neighborhood'].str.find('Not assigned')
a = a.tolist()
for i in range(len(a)):
    if a[i] == 0:
        ind.append(i)
for id in ind:
    df_Toronto.loc[id,'Neighborhood']=df_Toronto.loc[id,'Borough']


In [8]:
# Creating one Postal Code per Borough and grouping in the Neighborhood column all Neighborhoods in the same Postal Code 
df_Torontonew = df_Toronto.groupby(df_Toronto['PostalCode'],as_index=False).aggregate({'Borough': 'first', 'Neighborhood': lambda x: ', '.join(x)},sort=False)


In [9]:
df_Torontonew.shape

(103, 3)

# Including LAT and LON (Using CSV file)

### Reading CSV file

In [47]:
LatLon_df = pd.read_csv(r'C:\Users\v.hernandez.byd\Documents\Data Science\Geospatial_Coordinates.csv')
LatLon_df.head(10)
     

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


### Changing the name of column in LatLon_df and the merging Data Frames (df_Torontonew and LatLon_df)

In [49]:
LatLon_df = LatLon_df.rename(columns={'Postal Code': 'PostalCode'})
Toronto_data_df = pd.merge(df_Torontonew, LatLon_df, on='PostalCode')
Toronto_data_df.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
